In [15]:
%matplotlib inline
import random
import plotly.plotly as pl
from plotly.offline import iplot
import plotly.graph_objs as go


class Vector(list):
    def __init__(self, *args):
        if len(args) == 1:
            super().__init__(args[0])
        else:
            super().__init__(args)

    def __sub__(self, other):
        if len(self) != len(other):
            raise ValueError
        return Vector(self[i] - other[i] for i in range(len(self)))

    def __str__(self):
        return "(" + ", ".join(str(round(i, 3) or 0.0) for i in self) + ")"

    def copy(self):
        return Vector(self)


def hillclimb(p: Vector, f, steps=1000):
    step_size = [1] * len(p)
    acc = 1.2
    eps = 1e-16
    candidate = (-acc, -1 / acc, 0, 1 / acc, acc)
    for _ in range(steps):
        yield p.copy()
        val = f(p)
        for i in range(len(p)):
            best = -1
            best_score = -1e100
            for j in range(len(candidate)):
                delta = step_size[i] * candidate[j]
                p[i] += delta
                try:
                    temp = f(p)
                except ValueError:
                    continue
                finally:
                    p[i] -= delta
                if temp > best_score:
                    best_score = temp
                    best = j
            if candidate[best] == 0:
                step_size[i] /= acc
            else:
                step_size[i] *= candidate[best]
                p[i] += step_size[i]
        if 0 < f(p) - val < eps:
            return p
    return p


class func:
    arity = 1

    def __new__(cls, p: Vector):
        val = 1 - p[0]**2
        if isinstance(val, complex):
            raise ValueError
        return val


def starting(func, max_val=100):
    while True:
        p = Vector((random.random() - 0.5) * max_val for _ in range(func.arity))
        try:
            func(p)
        except ValueError:
            continue
        else:
            return p

        
start = starting(func, 10)
points = list(hillclimb(start, func))
vals = list(map(func, points))

In [56]:
# points = [p[0] for p in points]
func_graph_x = [-0.5 + 0.01*i for i in range(100)]
func_graph_y = list(map(lambda x: func([x]), func_graph_x))
tr1 = dict(x=func_graph_x, y=func_graph_y, mode='lines', line={'color': 'blue'})
frames = [
    {'data': [
        {
            'x': [points[i]],
            'y': [vals[i]],
            'mode': 'markers',
            'marker': {'color': 'red', 'size':10}
        }]} for i in range(1, 100)]
fig = {
    'layout': {
        'xaxis': {'range': [-1, 1], 'autorange': False},
        'yaxis': {'range': [min(vals)-0.125, max(vals) + 0.125], 'autorange': False},
        'title': 'Hill Climbing',
        'updatemenus': [{
            'buttons': [{
                'args': [None],
                'label': 'Play',
                'method': 'animate'
            }],
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons'
        }]
    },
    'data': [tr1, tr1],
    'frames': frames
}
pl.create_animations(fig)